# Interact with the Ftrace

There are multiple of interfaces that ftrace open to the user space and kernel space. For Ubuntu 22.04 distro, they reside in `/sys/kernel/debug/tracing/` or one of its subdirectory.

| Interface(File) | Description | User Space | Kernel Space |
|-----------------|-------------|------------|--------------|
| *trace* | show tracing information(Ftrace ring buffer). | `cat trace` | - |
| *current_tracer* | set/show current tracer being used. | `echo [tracer] > current_tracer`<br/>`cat current_tracer` |- |
| *tracing_on* | enable/disable trace recording. | `echo "0" > tracing_on`<br/>`echo "1" > tracing_on` | `void tracing_on(void);`<br/>`void tracing_off(void);` |
| *trace_marker* | write message into the Ftrace ring buffer. | `echo ... > trace_marker` | - |

There are utilities in [ftrace_utilites.sh](./ftrace_utilities.sh). The`find_debugfs` is used to find where the *debugfs* mounted. The `filter_trace` is used to filter specific message in *trace* file, because the output can be overwhelming on a busy system. Now, we source the utilities first:

In [1]:
source ./ftrace_utilities.sh
DEBUGFS="$(find_debugfs)"
TRACING="$DEBUGFS/tracing"

## 1. Write Debug Info from User Space to Ftrace Ring Buffer

The file *trace_marker* provides a way to write into the Ftrace ring buffer from user space. This marker will then appear in the *trace* file to give a location in the trace of where a specific event occurred.

In [2]:
echo 1 > $TRACING/tracing_on
echo "Hello, World!" > $TRACING/trace_marker
filter_trace 'Hello, World'

# tracer: nop
#
# entries-in-buffer/entries-written: 1/1   #P:8
#
#                                _-----=> irqs-off/BH-disabled
#                               / _----=> need-resched
#                              | / _---=> hardirq/softirq
#                              || / _--=> preempt-depth
#                              ||| / _-=> migrate-disable
#                              |||| /     delay
#           TASK-PID     CPU#  |||||  TIMESTAMP  FUNCTION
#              | |         |   |||||     |         |
           <...>-10078   [005] .....  5584.298864: tracing_mark_write: Hello, World!


And there is another example writen in C language, and the idea is the same: [Hello World in C](interact_with_ftrace/interact_with_ftrace.c).

In [6]:
make -C interact_with_ftrace > /dev/null
echo > $TRACING/trace # clear ring buffer
interact_with_ftrace/interact_with_ftrace.exe && \
filter_trace "Hello"
make clean -C interact_with_ftrace > /dev/null

# tracer: nop
#
# entries-in-buffer/entries-written: 1/1   #P:8
#
#                                _-----=> irqs-off/BH-disabled
#                               / _----=> need-resched
#                              | / _---=> hardirq/softirq
#                              || / _--=> preempt-depth
#                              ||| / _-=> migrate-disable
#                              |||| /     delay
#           TASK-PID     CPU#  |||||  TIMESTAMP  FUNCTION
#              | |         |   |||||     |         |
           <...>-10532   [007] .....  6528.161335: tracing_mark_write: Hello, Ftrace!
